In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime as DT
import sys
from csvvalidator import *
import os

pd.set_option('display.float_format', lambda x: '%.3f' % x)

ModuleNotFoundError: No module named 'seaborn'

In [76]:
#Validate data
field_names = (
               'Ticket number',
               'Issue Date',
               'Latitude',
               'Longitude',
               'Issue time',
               'Violation Description'
               )
validator = CSVValidator(field_names)
# basic header and record length checks
validator.add_header_check('EX1', 'bad header')
validator.add_record_length_check('EX2', 'unexpected record length')

NameError: name 'CSVValidator' is not defined

In [3]:
df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
0,4555162286,09/14/2022,1343,NaN,0.000,CA,202305,NaN,MAZD,PA,...,NaN,54,80.56E1,WHITE ZONE,58,6458719.509,1859071.049,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR
1,1120740482,01/30/2022,1030,NaN,NaN,CA,202103,NaN,RAM,TR,...,5A85,1,NaN,NaN,93,99999.000,99999.000,WESTERN,BLACK,TRAILER
2,1111081311,06/09/2022,808,NaN,NaN,CA,202101,NaN,TOYO,SU,...,00561,55,8056E4,NaN,93,6472422.966,1748169.243,55 - DOT - SOUTHERN,NaN,NaN
3,4555248633,09/06/2022,904,NaN,0.000,CA,201910,NaN,OTHR,PA,...,NaN,51,5204A-,DISPLAY OF TABS,25,6454803.494,1840042.963,51 - DOT - WESTERN,TAN,PASSENGER CAR
4,4549605012,05/26/2022,1656,NaN,0.000,CA,202211,NaN,HOND,PA,...,0402A,54,80.56E4+,RED ZONE,93,6472951.712,1846223.696,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR


In [4]:
#violation Description
assert pd.api.types.is_string_dtype(df['Violation Description'])
print('Violation Description is OK')

#ticket number
assert pd.api.types.is_string_dtype(df['Ticket number'])
print('Ticket number is OK')

#Lat/Lon
assert pd.api.types.is_float_dtype(df['Latitude'])
print('Latitude is OK')
assert pd.api.types.is_float_dtype(df['Longitude'])
print('Longitude is OK')

#Issue Time
assert pd.api.types.is_integer_dtype(df['Issue time'])
print('Issue time is OK')

Violation Description is OK
Ticket number is OK
Latitude is OK
Longitude is OK
Issue time is OK


In [5]:
#updating formatting so that I can translate issue date to datetime
df['Issue Date'] = df[df['Issue Date'].notnull()]['Issue Date'].apply(lambda x: x.split('T')[0])
df['Issue Date'] = pd.to_datetime(df['Issue Date'], infer_datetime_format=True)

#limiting dataset so it's easy to work with
today = pd.Timestamp('today').normalize()
df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
0,4555162286,2022-09-14,1343,NaN,0.000,CA,202305,NaN,MAZD,PA,...,NaN,54,80.56E1,WHITE ZONE,58,6458719.509,1859071.049,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR
1,1120740482,2022-01-30,1030,NaN,NaN,CA,202103,NaN,RAM,TR,...,5A85,1,NaN,NaN,93,99999.000,99999.000,WESTERN,BLACK,TRAILER
2,1111081311,2022-06-09,808,NaN,NaN,CA,202101,NaN,TOYO,SU,...,00561,55,8056E4,NaN,93,6472422.966,1748169.243,55 - DOT - SOUTHERN,NaN,NaN
3,4555248633,2022-09-06,904,NaN,0.000,CA,201910,NaN,OTHR,PA,...,NaN,51,5204A-,DISPLAY OF TABS,25,6454803.494,1840042.963,51 - DOT - WESTERN,TAN,PASSENGER CAR
4,4549605012,2022-05-26,1656,NaN,0.000,CA,202211,NaN,HOND,PA,...,0402A,54,80.56E4+,RED ZONE,93,6472951.712,1846223.696,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR


In [6]:
#pad anything that is less than 4 digits then isolate just the hours
df['Issue time'] = df['Issue time'].astype(str)
df['Issue time'] = df['Issue time'].apply(lambda x: x.split('.')[0])
df['Issue time'] = df[df['Issue time'].notnull()]['Issue time'].apply(lambda x: x.zfill(4))
df['Issue Hour'] = df[df['Issue time']!='0nan']['Issue time'].apply(lambda x: DT.datetime.strptime(x,'%H%M').hour)

#clean lat lon
df['Latitude'] = np.where(df['Latitude']==99999.000, np.nan, df['Latitude'])
df['Longitude'] = np.where(df['Longitude']==99999.000, np.nan, df['Longitude'])

#string for ticketnum
df['Ticket number'] = df['Ticket number'].astype(str)

In [7]:
#Updating the Lat Lon
import pyproj
pm = '+proj=lcc +lat_1=34.03333333333333 +lat_2=35.46666666666667 +lat_0=33.5 +lon_0=-118 +x_0=2000000 +y_0=500000.0000000002 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'
x1m,y1m = df['Latitude'].values, df['Longitude'].values
x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)
df['Latitude']=x2m
df['Longitude']=y2m

/data/homezvol2/ddlin/mambaforge-pypy3/envs/plotly/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ddlin/23122114/ipykernel_1357697/1413484023.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)


In [77]:
df = pd.read_csv("../cleaned_2022_parking_citation.csv")

/tmp/ipykernel_14191/960009943.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../cleaned_2022_parking_citation.csv")


In [78]:
df.columns

Index(['Ticket number', 'Issue Date', 'Issue time', 'Meter Id', 'Marked Time',
       'RP State Plate', 'Plate Expiry Date', 'VIN', 'Make', 'Body Style',
       'Color', 'Location', 'Route', 'Agency', 'Violation code',
       'Violation Description', 'Fine amount', 'Latitude', 'Longitude',
       'Agency Description', 'Color Description', 'Body Style Description',
       'Issue Hour'],
      dtype='object')

In [79]:
df['Violation Description'].unique()[:5]

array(['WHITE ZONE', nan, 'DISPLAY OF TABS', 'RED ZONE', 'METER EXP.'],
      dtype=object)

In [80]:
df['Issue time'].unique()[:5] 

array([1343, 1030,  808,  904, 1656])

In [81]:
sorted(df['Issue Hour'].unique())[:5]

[0, 1, 2, 3, 4]

In [82]:
df['Body Style Description'].unique()

array(['PASSENGER CAR', 'TRAILER', nan, 'TRUCK', 'PICK-UP TRUCK',
       'COMMERCIAL', 'VAN', 'MOTOR HOME', 'BUS', 'LIMOUSINE'],
      dtype=object)

In [83]:
df['Body Style'].unique()

array(['PA', 'TR', 'SU', 'TK', 'PU', 'CM', 'VN', 'MH', 'MS', 'MC', nan,
       'BO', 'BU', 'OT', 'RU', 'CP', 'RV', 'TL', 'LM', 'UT', '45', 'CR',
       'SE', 'JE', 'VA', 'SV', 'CO', 'CC', 'TC', 'SH', 'TO', '4D', 'HS',
       'CA', 'CV', 'SW', '2D', 'SD', 'DS', 'LI', 'SC', 'RB', 'US', 'TT',
       '02', 'PK', 'WK', 'WG', 'MO', 'VC', 'DU', 'FB', 'RX', 'VS', 'BX',
       'VT', '4H', 'ED', 'YY', 'HC', 'DP', 'BT', 'UP', '4W', 'RI', 'SL',
       'NX', 'CB', 'VE', '4C', 'VH', 'WH', 'SR', 'HB', 'WA', 'SB', 'M4',
       'NR', 'ST', 'MP', 'I1', '2T', 'F7', 'I5', '2H', 'TA', 'EC', 'MV',
       'XO', 'TV', 'RA', 'P3', '01', '62', 'E3', 'VP', '95', 'AU'],
      dtype=object)

In [84]:
df = df[[ 'RP State Plate', 'Make', 'Body Style', 'Color Description', 'Agency Description', 'Issue Hour', 'Latitude', 'Longitude', 'Violation Description']]
df.shape

(2014821, 9)

In [85]:
df = df[~df.isna().any(axis=1)]

In [86]:
from sklearn.cluster import KMeans

# Assuming your dataframe is named 'data' and has columns 'Latitude' and 'Longitude'
coordinates = df[['Latitude', 'Longitude']].values

# Create a KMeans clustering model with 100 clusters
kmeans = KMeans(n_clusters=100, random_state=0).fit(coordinates)

# Add cluster labels to the original dataframe
df['Cluster'] = kmeans.labels_

/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [87]:
df

,RP State Plate,Make,Body Style,Color Description,Agency Description,Issue Hour,Latitude,Longitude,Violation Description,Cluster
0,CA,MAZD,PA,GREY,54 - DOT - HOLLYWOOD,13,-118.340055,34.100336,WHITE ZONE,8
3,CA,OTHR,PA,TAN,51 - DOT - WESTERN,9,-118.352770,34.048013,DISPLAY OF TABS,6
4,CA,HOND,PA,GREY,54 - DOT - HOLLYWOOD,16,-118.292919,34.065156,RED ZONE,89
5,CA,TOYT,PA,BLACK,53 - DOT - VALLEY,14,-118.374401,34.168097,METER EXP.,84
7,TX,DODG,PA,BLACK,56 - DOT - CENTRAL,16,-118.249925,34.045272,METER EXP.,83
...,...,...,...,...,...,...,...,...,...,...
2014816,CA,VOLK,PA,BLACK,51 - DOT - WESTERN,16,-118.467913,33.991691,METER EXP.,76
2014817,CA,MERZ,PA,WHITE,54 - DOT - HOLLYWOOD,17,-118.346190,34.097976,NO STOPPING/ANTI-GRIDLOCK ZONE,8
2014818,CA,HOND,PA,BLACK,53 - DOT - VALLEY,16,-118.288133,33.738757,STOP/STAND PROHIBIT,4
2014819,CA,TOYT,PA,BLACK,51 - DOT - WESTERN,16,-118.349414,34.062276,METER EXP.,33


In [88]:
df.to_csv('../cleaned_2022_parking_citation2.csv', index=False)


In [90]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

# Check cuda
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)


# 1. Data Preparation
# ---------------------
data = pd.read_csv("../cleaned_2022_parking_citation2.csv")
features = [ 'RP State Plate', 'Make', 'Body Style', 'Color Description', 'Agency Description', 'Issue Hour', 'Cluster']
X = data[features]

label_encoders = {}
for col in ['RP State Plate', 'Make', 'Body Style', 'Color Description', 'Agency Description', 'Cluster']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# One-hot encode 'Issue Hour'
ohe = OneHotEncoder(sparse=False, dtype=int)
issue_hour_encoded = ohe.fit_transform(X[['Issue Hour']])
issue_hour_df = pd.DataFrame(issue_hour_encoded, columns=[f"IssueHour_{int(i)}" for i in range(issue_hour_encoded.shape[1])])

# Reset the index of X before concatenation
X.reset_index(drop=True, inplace=True)

# Replace the original 'Issue Hour' column with the one-hot encoded data
X = pd.concat([X.drop(columns='Issue Hour'), issue_hour_df], axis=1)


y = LabelEncoder().fit_transform(data['Violation Description'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
batch_size = 256
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 2. Model Definition
# ---------------------
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

model = SimpleNN(X_train.shape[1], len(set(y_train))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

# 3. Training
# ------------
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    # 4. Evaluation
    # ---------------
    with torch.no_grad():
        y_pred = model(X_train_tensor)
        _, predicted = torch.max(y_pred, 1)
        accuracy = (predicted == y_train_tensor).sum().item() / len(y_train_tensor)
        print(f"Training Accuracy: {accuracy}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time} seconds")

True
cuda:0


/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10, Loss: 1.9293214082717896
Training Accuracy: 0.41698695931874435
Epoch 2/10, Loss: 1.6889859437942505
Training Accuracy: 0.42009175362271434
Epoch 3/10, Loss: 2.048356294631958
Training Accuracy: 0.42744758104085573
Epoch 4/10, Loss: 1.808008074760437
Training Accuracy: 0.4295431127539171
Epoch 5/10, Loss: 1.8518352508544922
Training Accuracy: 0.42972574771297417
Epoch 6/10, Loss: 1.8611969947814941
Training Accuracy: 0.43094013389595803
Epoch 7/10, Loss: 1.907226800918579
Training Accuracy: 0.43274058748487126
Epoch 8/10, Loss: 2.0424554347991943
Training Accuracy: 0.43207001733669165
Epoch 9/10, Loss: 1.5820761919021606
Training Accuracy: 0.43386570060951013
Epoch 10/10, Loss: 1.8171765804290771
Training Accuracy: 0.43538402407510385
Total execution time: 165.24951720237732 seconds


In [50]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# import pandas as pd

# # Load your data
# data = pd.read_csv("cleaned_2022_parking_citation.csv")

# # Preprocess the data (similar to what we did above)
# data = data.dropna(subset=['Violation Description'])
# features = ['RP State Plate', 'Make', 'Body Style', 'Color Description', 'Agency Description', 'Issue Hour']
# X = data[features]
# X = X.fillna('Unknown')

# label_encoders = {}
# for col in X.select_dtypes(include=['object']).columns:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     label_encoders[col] = le

# scaler = StandardScaler()
# X[['Issue time', 'Issue Hour']] = scaler.fit_transform(X[['Issue time', 'Issue Hour']])
# y = LabelEncoder().fit_transform(data['Violation Description'])

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the XGBoost classifier
# clf = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y_train)), random_state=42, use_label_encoder=False)
# clf.fit(X_train, y_train, eval_metric='mlogloss')

# # Get training accuracy (you can also get testing accuracy if you wish)
# train_accuracy = clf.score(X_train, y_train)
# print(f"Training Accuracy: {train_accuracy}")